### Auswertungsskizze für die Bewertung von möglichen Superblock-Gebieten anhand Kriterien einer 15-minuten-Stadt

Das Notebook skizziert beispielhaft, wie die Bewertung von potenziell spannenden Stadtgebieten für das errichten von Superblocks aussehen kann. Hierzu werden Stadtgebiete anhand der Methode von Eggiman identifiziert, in denen infrastrukturelle Gegebenheiten die einrichtung von Verkehrsberuhigten Zonen ermöglichen. In einem Zweiten Schritt werden Open Street Map Daten genutzt, um Einrichtungen des täglichen Bedarfs (Points of Interest - POI) innerhalb identifizierter Gebiete zu zählen. Die Anzahl der zugänglichen Einrichtungen werden genutzt, um eine Bewertung lebensqualitätsbezogener Kriterien in einem Netzdiagramm darzustellen. Das Vorgehen ist skizzenhaft als Vorschlag gedacht und ist Platzhalter für eine wissenschaftlich fundierte und systematische Auswertung. In diesem Zuge wird auf die aktuellen Arbeiten von Moritz Kreuschner der TU Berlin verwiesen, der zu eienr solchen systematsichen Auswertung forscht.

##### 1. Packages und Daten einlesen:

In [ ]:
import json 
import os
import folium
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
import plotly.express as px
from helper_functions import add_categorical_legend

# Read data from Eggimann code result into geopandas:
input_file=json.load(open("data/blocks.json", "r", encoding="utf-8"))

input_file = json.loads(f"""
 {input_file}
""")

gdf = gpd.GeoDataFrame.from_features(input_file["features"])
gdf = gdf.set_crs("EPSG:4326")

# Read Results from Open Street Map Queries:
path_to_json = './json_osm/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
json_names = [x[:-5] for x in json_files]
osm_results = {}
for file,name in zip(json_files, json_names):
   with open(path_to_json+file, 'r') as f:
      osm_results[name] = json.load(f)

# Exclude Spielplätz for simple example
del osm_results['Spielplätze']

##### 2. Visualisierung von drei Stadtgebieten als Beispiel:

In [24]:
# Filter out rectangular area of interest around Seeburgviertel 
# between lat (12.375, 12395) and lon (51.325, 51.35)
grid_polygon = Polygon([(12.375, 51.325), (12.375, 51.35), (12.395, 51.35), (12.395, 51.325)])
match_indices = gdf.sindex.query(grid_polygon, predicate="contains")
geo_df_filtered = gdf.iloc[match_indices]
# Filter out the largest identified potential superblocks/miniblocks
geo_df_filtered = geo_df_filtered[geo_df_filtered["area"]>200000]

In [12]:
# Initiate Folium Map to display results for rectangular area of interest
m = folium.Map(location=[51.33, 12.38], zoom_start=15, tiles="CartoDB positron")
style = {'fillColor': '#00FFFFFF', 'fillOpacity': 0.01}
for _, r in geo_df_filtered.iterrows():
    geo_j = folium.GeoJson(data=geo_df_filtered["geometry"], style_function=lambda x : style)
    folium.Popup(r["b_type"]).add_to(geo_j)
    geo_j.add_to(m)

In [14]:
# Select only results from Open Street Map in area of interest:
osm_seeburg = {}
for result_osm in osm_results:
    dict_obj_json = osm_results[str(result_osm)]["elements"]

    new_data = [ v for v in dict_obj_json if (v['center']['lat'] >=51.326 and v['center']['lat'] <= 51.345) and (v['center']['lon'] >= 12.375) and  (v['center']['lon'] <= 12.395 )]
    osm_seeburg[str(result_osm)] = new_data


In [15]:
# Plot results from Open Street Map query in different colors by criteria
for result_osm in osm_results:

    if result_osm in ['Bushaltestellen', 'OEPNV']:
        marker_col = 'gray'
        name_icon = 'Mobilität'
    if result_osm in ['Kino', 'Theater']:
        marker_col = 'purple'
        name_icon = 'Freizeit'
    if result_osm in ['Kindergarten', 'Grundschulen', 'Bibliotheken']:
        marker_col = 'blue'
        name_icon = 'Bildung'
    if result_osm in ['Parks']:
        marker_col = 'green'   
        name_icon = 'Naherholung'         
    if result_osm in ['Bäckereien_1', 'Bäckereien_2', 'Restaurants', 'Supermärkte']:
        marker_col = 'black' 
        name_icon = 'Nahversorgung'  
    if result_osm in ['Ärzte', 'Krankenhäuser']:
        marker_col = 'red'    
        name_icon = 'Gesundheit'       
    for facility in osm_seeburg[str(result_osm)]:
               
        #Setup the content of the popup
        iframe = folium.IFrame(name_icon )#+ aerzte["elements"][i]['tags']["addr:street"] + aerzte["elements"][i]['tags']['addr:housenumber'])
        
        #Initialise the popup using the iframe
        popup = folium.Popup(iframe, min_width=80, max_width=100)
        
        #Add each row to the map
        folium.Marker(location=[facility['center']['lat'],facility['center']['lon']],
                      popup = popup, icon=folium.Icon(color=marker_col, icon='')).add_to(m)

m = add_categorical_legend(m, 'Legende',
                             colors = ['#575757','#d152b8', '#38aadd', '#72ae26', '#303030', '#d63e2a'],
                           labels = ['Mobilität: Bushaltestellen, ÖPNV', 'Freizeit: Kino, Theater', 'Bildung: Kindergarten, Grundschulen, Bibliotheken', 
                           'Naherholung: Parks', 'Nahversorgung: Bäckereien, Restaurants, Supermärkte', 'Gesundheit: Ärztinnen, Krankenhäuser'])
m

##### 3. Auswertung der Open Street Map Daten (Skizze):

In [31]:
# Evaluate number of points of interests (poi) for visualisation in Netdiagramm
summe_nahversorgung = [f"{poi}: {len(osm_seeburg[poi])}" for poi in osm_seeburg if poi in ['Bäckereien_1', 'Bäckereien_2', 'Restaurants', 'Supermärkte']]
print(summe_nahversorgung)

summe_freizeit = [f"{poi}: {len(osm_seeburg[poi])}" for poi in osm_seeburg if poi in ['Kino', 'Theater']]
print(summe_freizeit)

summe_bildung = [f"{poi}: {len(osm_seeburg[poi])}" for poi in osm_seeburg if poi in ['Kindergarten', 'Grundschulen', 'Bibliotheken']]
print(summe_bildung)

summe_naherholung = [f"{poi}: {len(osm_seeburg[poi])}" for poi in osm_seeburg if poi in ['Spielplätze', 'Parks']]
print(summe_naherholung)

summe_gesundheit = [f"{poi}: {len(osm_seeburg[poi])}" for poi in osm_seeburg if poi in ['Ärzte', 'Krankenhäuser']]
print(summe_gesundheit)


['Bäckereien_2: 1', 'Restaurants: 6', 'Supermärkte: 0', 'Bäckereien_1: 5']
['Kino: 0', 'Theater: 1']
['Kindergarten: 9', 'Bibliotheken: 1', 'Grundschulen: 2']
['Parks: 25']
['Ärzte: 2', 'Krankenhäuser: 1']


Logik hinter der Kodierung von Häufigkeiten verschiedeneer Einrichtungen:
- Max Punktzahl = 5, wenn es mehr als 2 Einrichtungen pro Kategorie gibt wird die max. Punktzahl erreicht.
- Öffentliche Einrichtugnen: Max Punktzahl bei mind. eine Einrichtung pro Kategorie.
Abzüge der vollen Punktzahl:
- bei weniger als 2 Einrichutungen (privatwirtschaftlich) pro Kategorie -> -0.5
- bei keiner Einrichtung pro Kategorie
- solang keine einzige Kategorie (privatwirtschaftlich) mehr als 1 Einrichtung hat

Bei Fragen zu einer wissenschaftlich fundierten Auswertung siehe Kontakt Moritz Kreuschner von der TU Berlin

In [17]:
# Coding Values for visualization in Netdiagramm using number of available facilities:
df = pd.DataFrame(dict(
    r=[5, 1, 5, 5, 3, 5],
    theta=['Mobilität', 'Freizeit', 'Bildung', 'Naherholung', 'Nahversorgung', 'Gesundheit'],
    color_pallette = ['skyblue','skyblue','skyblue','skyblue','skyblue','skyblue']))
fig = px.line_polar(df, r='r', theta='theta', line_close=True)
fig.update_traces(line_color='#3d9fd0', line_width=5)
fig.show()


/home/antonia/anaconda3/envs/pycon_demo/lib/python3.10/site-packages/plotly/express/_core.py:271: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trace_data = trace_data.append(trace_data.iloc[0])
